# Proyecto 1 de Simulación

## Dependencias:

In [4]:
import queue
import heapq
import random
import numpy as np

In [5]:
class Service:
    def __init__(self, id, service_time_distribution):
        self.id = id
        self.service_time_distribution = service_time_distribution
        self.line = queue.Queue()
        self.available = True
        self.time = 0
        self.time_available = 0
        self.time_working = 0
        self.clients_served = 0
    
    def update_time(self, time):
        delta = time - self.time
        if self.available:
            self.time_available = self.time_available + delta
        else:
            self.time_working = self.time_working + delta
        self.time = time

    def lineup(self, person_id, time):
        self.update_time(time)
        self.line.put(person_id)
    
    def serve(self, person_id, time):
        self.update_time(time)
        self.available = False
        service_time = self.service_time_distribution.random()
        return (self.time + service_time, 'FINISHED_SERVICE', person_id, self.id)
    
    def finish_service_to(self, person_id, time):
        self.update_time(time)
        self.available = True
        self.clients_served = self.clients_served + 1
    
    def move_line(self, time):
        self.update_time(time)
        if not self.line.empty():
            return self.line.get()
        return None

In [6]:
class Person:
    def __init__(self, id, time):
        self.id = id
        self.time = time
        self.time_waiting = 0
        self.time_being_served = 0
        self.services_visited = 0
        self.waiting = True
    
    def time_update(self, time):
        delta = time - self.time
        if self.waiting:
            self.time_waiting = self.time_waiting + delta
        else:
            self.time_being_served = self.time_being_served + delta
        self.time = time

    def lineup(self, service_id, time):
        self.time_update(time)
        self.waiting = True
    
    def be_served(self, service_id, time):
        self.time_update(time)
        self.waiting = False

    def finish_service_from(self, service_id, time):
        self.time_update(time)
        self.waiting = True

In [7]:
class Serial_Servers_Sim:
    def __init__(self, arrivals_distribution, service_time_distributions):
        self.n_of_servers = len(service_time_distributions)
        self.arrivals_distribution = arrivals_distribution
        self.service_time_distributions = service_time_distributions
        self.services = None
        self.timeline = []
        self.persons = []
    
    def simulate(self, time_to_simulate):
        self.services = [Service(i, self.service_time_distributions[i]) for i in range(self.n_of_servers)]
        self.compute_arrivals_until(time_to_simulate)
        heapq.heapify(self.timeline)
        while len(self.timeline):
            time, operation, person_id, service_id = heapq.heappop(self.timeline)
            match operation:
                case 'ARRIVAL':
                    self.service_solicitude(0, person_id, time)
                case 'FINISHED_SERVICE':
                    self.service_finished(service_id, person_id, time)
                    self.move_service_line(service_id, time)
                    if service_id + 1 < self.n_of_servers:
                        self.service_solicitude(service_id + 1, person_id, time)
    
    def service_solicitude(self, service_id, person_id, time):
        service = self.services[service_id]
        person = self.persons[person_id]
        if service.available:
            new_op = service.serve(person_id, time)
            heapq.heappush(self.timeline, new_op)
            person.be_served(service_id, time)
        else:
            service.lineup(person_id, time)
            person.lineup(service_id, time)
    
    def service_finished(self, service_id, person_id, time):
        service = self.services[service_id]
        person = self.persons[person_id]
        service.finish_service_to(person_id, time)
        person.finish_service_from(service_id, time)
    
    def move_service_line(self, service_id, time):
        service = self.services[service_id]
        person_id = service.move_line(time)
        if person_id:
            new_op = service.serve(person_id, time)
            heapq.heappush(self.timeline, new_op)
            person = self.persons[person_id]
            person.be_served(service_id, time)
    
    def compute_arrivals_until(self, time_to_simulate):
        time = 0
        person_id = 0
        while time < time_to_simulate:
            time = time + self.arrivals_distribution.random()
            self.timeline.append((time, 'ARRIVAL', person_id, 0))
            self.persons.append(Person(person_id, time))
            person_id = person_id + 1

In [8]:
def uniform_var(max = 5):
    return np.random.uniform(0,max,None)
def normal_var(loc = 3.0,scale = 1.0):
    return np.random.normal(loc,scale,None)
def exponential_var(scale = 1.0):
    return  np.random.exponential(scale,None)
def gamma_var(shape = 1.0,scale = 1.0):
    return np.random.gamma(shape,scale,None)
def weibull(shape = 1):
    return np.random.weibull(shape,None)
def laplace_var(loc = 3.0,scale = 1.0):
    return np.random.laplace(loc,scale,None)
def rayleigh_var(scale = 1.0):
    return np.random.rayleigh(scale,None)
def beta_var(a = 2,b = 5):
    return np.random.beta(a,b,None)

In [9]:
class Distribution_Example:
    def __init__(self, distribution = random.random) -> None:
        self.distribution = distribution
    def random(self):
        return self.distribution()

def ejecute_simulation(distribution,time = 10, n = 1000):
    print(f""" Simulation using {distribution.__name__}""")
    dates = [[],[],[],[]]
    dates_name = ["Tiempo de espera por persona", 
                  "Tiempo inactivo por servidor",
                  "Tiempo activo por servidor",
                  "Personas atendidas por servidor"]
    for i in range(1,n):
        simulation = Serial_Servers_Sim(Distribution_Example(distribution), [Distribution_Example(distribution)])
        simulation.simulate(time)
        current_dates = calculate_person_dates(simulation)
        current_dates.extend(calculate_services_dates(simulation))
        update_dates(dates,current_dates)
    print_result(dates,dates_name)

def ave(l):
    return sum(l)/len(l)     

def print_result(dates,dates_name):
    print(f"""Datos recopilados:""")
    for i in range(0,len(dates)):
        print(f"""{dates_name[i]}: Promedio: {ave(dates[i])}
                            Varianza: {np.var(dates[i])}""")
        
def update_dates(dates,current_dates):
    for i in range(0,len(dates)):
        dates[i].append(current_dates[i])

def calculate_person_dates(simulation):
    time_waitings = []
    for person in simulation.persons:
        time_waitings.append(person.time_waiting)
    return [ave(time_waitings)]


def calculate_services_dates(simulation):
    time_availables = []
    time_workings = []
    clients_serveds = []
    for service in simulation.services:
        time_availables.append(service. time_available)    
        time_workings.append(service.time_working)
        clients_serveds.append(service.clients_served)
    return [ave(time_availables),ave(time_workings),ave(clients_serveds)]

distributions = [uniform_var,normal_var,exponential_var,
                 gamma_var,weibull,laplace_var,rayleigh_var, beta_var]

for d in distributions:
    ejecute_simulation(d)


 Simulation using uniform_var
Datos recopilados:
Tiempo de espera por persona: Promedio: 1.3342635863801118
                            Varianza: 2.4183245645544593
Tiempo inactivo por servidor: Promedio: 4.467611952266192
                            Varianza: 5.79801311536032
Tiempo activo por servidor: Promedio: 11.783298794426205
                            Varianza: 20.76827009422569
Personas atendidas por servidor: Promedio: 4.694694694694695
                            Varianza: 1.5854673492311133
 Simulation using normal_var
Datos recopilados:
Tiempo de espera por persona: Promedio: 0.6789673351628521
                            Varianza: 0.6552392615992128
Tiempo inactivo por servidor: Promedio: 4.243648601578753
                            Varianza: 2.3698749298432773
Tiempo activo por servidor: Promedio: 11.570618971256218
                            Varianza: 8.162720699351535
Personas atendidas por servidor: Promedio: 3.87987987987988
                            Varianza: 0